In [184]:
import pandas as pd
df=pd.read_csv("movies.csv")
df.head(5)

,title,overview,genres
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"Drama, Crime"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama, Crime"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"Drama, Crime"
3,Schindler's List,The true story of how businessman Oskar Schind...,"Drama, History, War"
4,12 Angry Men,The defense and the prosecution have rested an...,Drama


In [185]:
df["text"]=df["title"]+ " " + df["overview"]
df.drop(columns=["title","overview"],inplace=True)

In [186]:
import re
import pandas
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
nltk.download('punkt')
nltk.download('stopwords')


stop_words=set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # Lowercase
    text=str(text).lower()
    # Remove html tags
    text=re.sub(r"<.*?>"," ", text)
    # Remove URL
    text=re.sub(r"http\S+|www\S+|https\S+"," ",text)
    # Remove Punctuation
    text=text.translate(str.maketrans("","",string.punctuation))
    # Tokenize
    words=word_tokenize(text)
    # Stopwords Removal
    words=[word for word in words if word not in stop_words] 
    # Lemmatization
    words=[lemmatizer.lemmatize(word) for word in words]
    # join back to string
    return " ".join(words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\R.C\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\R.C\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [187]:
#Apply preprocessing
df['text']=df["text"].apply(preprocess_text)

In [188]:
df.isnull().sum()

genres    3
text      0
dtype: int64

In [189]:
df["genres"]=df["genres"].fillna("")

In [190]:
df.isnull().sum()

genres    0
text      0
dtype: int64

In [191]:
X=df["text"]
binarizer=MultiLabelBinarizer()
y=binarizer.fit_transform(df["genres"])


In [192]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=0)

In [193]:
vectorizer=TfidfVectorizer(max_features=5000)
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [196]:
model=OneVsRestClassifier(LogisticRegression(max_iter=2000))
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [198]:
from sklearn.metrics import classification_report
# predict porbabilities
y_pred_prob=model.predict_proba(X_test)
# Convert to binary prediction
y_pred = (y_pred_prob >= 0.5).astype(int)
print(classification_report(y_test, y_pred, target_names=binarizer.classes_))


              precision    recall  f1-score   support

                   0.87      1.00      0.93      1481
           ,       0.87      1.00      0.93      1480
           A       0.82      0.58      0.68       594
           C       0.72      0.67      0.69       792
           D       0.71      0.71      0.71       855
           F       0.86      0.46      0.60       465
           H       0.86      0.13      0.23       327
           M       0.73      0.04      0.07       212
           R       0.81      0.24      0.37       301
           S       0.96      0.23      0.37       186
           T       0.73      0.31      0.43       475
           V       0.00      0.00      0.00        16
           W       1.00      0.08      0.15        84
           a       0.75      0.94      0.83      1200
           c       0.74      0.62      0.67       810
           d       0.69      0.63      0.66       766
           e       0.83      1.00      0.90      1410
           h       0.74    

e:\Conda\envs\Machine_Learning\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
e:\Conda\envs\Machine_Learning\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
